<!-- Header with style -->
<h1 style="font-size:50px; color:blue; background-color:lightgreen; text-align:center;">MOVIE RECOMMENDATION SYSTEM</h1>


# why is this useful for a recommendation system ?
* Feature Reduction : Keeping only the top 3 actors reduces noise and makes similarity calculations more efficient.
* Better Performance: improves the effictiveness of content based filtering by focusing on key cast members.
* Vectorization Readiness: Helps in converting categorical text data into numerical vectors for machine learning.

# Steps 
* 1.import libraies
* 2.import the dataset
* 3.data analysis -DE,DM,DC,DV,EDA[OPTIONAL] hyper parameter tuning
* 4.feature engineering encoders,feature scaling
* 5.split the data into two sets using the CV
* 6.model selection
* 7.training the model
* 8.test the model
* 9.performance -confusion metric

importing the libraries

In [4]:
import os
print(os.getcwd())  # Shows the directory where Jupyter Notebook started

C:\Users\lavan\Downloads\vantharr data science study materials\PROJECTS\Movie recomendation project


In [5]:
import numpy as np
import pandas as pd
import json
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler


* re is python's regular expression (regex)module.it allows you to work with text by searching,matching,and manipulating patterns
* Data Manipulation: Libraries like Pandas, NumPy, and JSON are essential for data loading, processing, and manipulation.

* Visualization: Matplotlib and Seaborn are used to create visualizations to understand the dataset and results better.

* Machine Learning: KNeighborsClassifier is the classifier for performing K-Nearest Neighbors (KNN) classification. The code also imports functions for evaluating model performance (confusion_matrix, accuracy_score) and techniques for cross-validation (StratifiedKFold, KFold).

* Data Preprocessing: StandardScaler is used to scale the features before training machine learning models to ensure that all features are on the same scale.

# key features of re module
* pattern matching - find specific text patterns in a string
* searching & extracting - locate and extract specific substrings
* replacing text - modify strings based on patterns
* spliting strings - break text into parts using delimiters

In [6]:
#Load the datasets
movies = pd.read_csv(r"C:\Users\lavan\Downloads\vantharr data science study materials\PROJECTS\Movie recomendation project\tmdb_5000_movies.csv")
credits=pd.read_csv(r"C:\Users\lavan\Downloads\vantharr data science study materials\PROJECTS\Movie recomendation project\tmdb_5000_credits.csv")

# Data analysis

In [7]:
movies.head(3) #it shows the first 3 rows from the movie dataset 

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [8]:
credits.head(3) #it shows the first 3 rows from the credits dataset 

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [9]:
movies.describe() #this function describes Statistical Summary of the dataset

,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [10]:
credits.describe()

,movie_id
count,4803.000000
mean,57165.484281
std,88694.614033
min,5.000000
25%,9014.500000
50%,14629.000000
75%,58610.500000
max,459488.000000


In [11]:
movies.info() #this function gives the information about the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [12]:
movies.columns #this function shows the columns present in the dataset

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [13]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [14]:
(movies.genres[0]) #it shows the genres column's "0" row contains data

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [15]:
credits.describe(include="object") # statistical calculation for categorical data

,title,cast,crew
count,4803,4803,4803
unique,4800,4761,4776
top,The Host,[],[]
freq,2,43,28


In [16]:
credits.describe(include ="all") #statistical calculations for all columns of data

,movie_id,title,cast,crew
count,4803.000000,4803,4803,4803
unique,NaN,4800,4761,4776
top,NaN,The Host,[],[]
freq,NaN,2,43,28
mean,57165.484281,NaN,NaN,NaN
std,88694.614033,NaN,NaN,NaN
min,5.000000,NaN,NaN,NaN
25%,9014.500000,NaN,NaN,NaN
50%,14629.000000,NaN,NaN,NaN
75%,58610.500000,NaN,NaN,NaN


In [17]:
(movies.genres[0])

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [18]:
re.findall('name',movies.genres[0])
#re.findall() is a function from the re module, which is used to find all non-overlapping occurrences of a pattern in a given string. 
#It returns a list of all matches.
#The pattern 'name' is a string that we're looking for in the input text. It means we want to search for the literal string 'name'.
# in genres column contains 4 types of name rows

['name', 'name', 'name', 'name']

In [19]:
a = eval(movies.genres[0])

In [20]:
for i in range(len(a)):
    print(a[i]['name'])

Action
Adventure
Fantasy
Science Fiction


* eval(): This is a built-in Python function that takes a string expression and evaluates it as a Python expression. It will execute the string as if it were a Python code snippet.
For example, if the string is '[1, 2, 3]', eval() will evaluate it as the Python list [1, 2, 3].
In this case, eval(movies.genres[0]) is executing the first element of movies.genres as a Python expression.

* The code first evaluates the string from movies.genres[0] to turn it into a list of dictionaries.
Then, it iterates through this list, and for each dictionary, it accesses the value associated with the 'name' key and prints it.

# extract information from a json file , and create a list

In [21]:
import ast

for i, k in zip(movies.genres, range(len(movies.genres))):
    if isinstance(i, str):  # Ensure i is a string
        genres_list = ast.literal_eval(i)  # Safely evaluate the string
        movies.at[k, 'genres'] = [genre['name'] for genre in genres_list]
    else:
        movies.at[k, 'genres'] = []  # Handle non-string values (e.g., NaN)


This code transforms and cleans the genres column of the movies DataFrame by doing the following:
* Iterates over each entry in the movies.genres column (each genre list or string).
* If the genre data is a string, it uses ast.literal_eval() to safely parse it into a Python list of dictionaries.
* Extracts the 'name' field from each dictionary and replaces the original string in movies.genres with a list of genre names (e.g., ['Action', 'Drama']).
* If the genre data is not a string (e.g., NaN or already a list), it assigns an empty list to that row in the genres column.

In [22]:
movies.genres

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4798                        [Action, Crime, Thriller]
4799                                [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV Movie]
4801                                               []
4802                                    [Documentary]
Name: genres, Length: 4803, dtype: object

In [23]:
# movies.keywords[0]
import ast

for i, k in zip(movies.keywords, range(len(movies.keywords))):
    keywords_list = ast.literal_eval(i)  # Safely convert string to list of dicts
    movies.at[k, 'keywords'] = [keyword['name'] for keyword in keywords_list]


* it processes the keywords column in the movies DataFrame. The goal is to convert the string representation of a list of dictionaries (in the keywords column) into a list of keyword names
1. Iterate through each entry in movies.keywords:
2. For each row, it checks the string representation of a list of dictionaries in the keywords column.
3. Use ast.literal_eval() to safely evaluate the string into an actual Python list of dictionaries.
4. Extract the 'name' field from each dictionary in the list, and create a new list of these names (e.g., ['Action', 'Drama']).
5. Assign the new list of names back to the keywords column for that row in the DataFrame.

This code is useful when:
* The keywords column contains a string representation of a list of dictionaries, and you need to convert it into a list of relevant values (in this case, the 'name' field from each dictionary).
* It safely parses the string using ast.literal_eval() instead of eval() to avoid any potential security risks.
* The code is flexible enough to handle various formats, such as empty lists (which will result in an empty list in the final result).

In [24]:
movies.keywords[0]  # The expression movies.keywords[0] is used to access the first element in the keywords 
# column (or attribute) of the movies DataFrame or object.

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d']

In [25]:
# movies.production_companies[0]
import ast

for i, k in zip(movies.production_companies, range(len(movies.production_companies))):
    if isinstance(i, str):  # Ensure the value is a string
        companies_list = ast.literal_eval(i)  # Safely evaluate the string
        movies.at[k, 'production_companies'] = [company['name'] for company in companies_list]
    else:
        movies.at[k, 'production_companies'] = []  # Handle non-string values


* Iterates over the production_companies column of the movies DataFrame, where each entry may be a string representation of a list of dictionaries.
* Checks if the value is a string. If it is:
1. It safely converts the string to a list of dictionaries using ast.literal_eval().
2. It then extracts the 'name' field from each dictionary and creates a list of company names (e.g., ['Company A', 'Company B']).
3. This list of company names is then assigned back to the production_companies column for that movie.
4. If the value is not a string, it assigns an empty list ([]) to the production_companies column for that row. This handles missing or non-string values.

* same for the production_countries,spoken_language, columns in the movie dataset

In [26]:
# movies.production_countries[0]
import ast

for i, k in zip(movies.production_countries, range(len(movies.production_countries))):
    if isinstance(i, str):  # Check if the value is a string
        countries_list = ast.literal_eval(i)  # Safely evaluate the string
        movies.at[k, 'production_countries'] = [country['name'] for country in countries_list]
    else:
        movies.at[k, 'production_countries'] = []  # Handle non-string values


In [27]:
# movies.spoken_languages[0]
import ast

for i, k in zip(movies.spoken_languages, range(len(movies.spoken_languages))):
    if isinstance(i, str):  # Check if the value is a string
        spoken_languages = ast.literal_eval(i)  # Safely evaluate the string
        movies.at[k, 'spoken_languages'] = [language['name'] for language in spoken_languages]
    else:
        movies.at[k, 'spoken_languages'] = []  # Handle non-string values

In [28]:
movies = movies.merge(credits, on = "title")   #The code snippet movies = movies.merge(credits, on = "title") merges two DataFrames: 
#movies and credits, based on the common column "title"

In [29]:
movies.head()  #it displays the first 5 rows from the dataset

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...",...,162.0,"[English, Español]",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",...,169.0,[English],Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[spy, based on novel, secret agent, sequel, mi...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[Columbia Pictures, Danjaq, B24]",...,148.0,"[Français, English, Español, Italiano, Deutsch]",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[Action, Crime, Drama, Thriller]",http://www.thedarkknightrises.com/,49026,"[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[Legendary Pictures, Warner Bros., DC Entertai...",...,165.0,[English],Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[Action, Adventure, Science Fiction]",http://movies.disney.com/john-carter,49529,"[based on novel, mars, medallion, space travel...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,[Walt Disney Pictures],...,132.0,[English],Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [30]:
movies = movies[["movie_id", "title", "overview", "genres", "keywords", "cast", "crew"]]
# The code movies = movies[["movie_id", "title", "overview", "genres", "keywords", "cast", "crew"]] is used to 
#select specific columns from the movies DataFrame and reassign this subset of columns back to the movies variable

In [31]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [32]:
import ast

The ast(abstract syntax tree) module in python is used to process and analyze python code by converting it into an abstract syntax tree(AST).
This allows developers to parse, modify, and analyze python code programmatically.

In [33]:
def convert(text):
    L=[] # L is the empty list
    for i in ast.literal_eval(text):
        L.append(i["name"])
    return L

This function helps convert those string based JSON Objects into a usable python list.
It is especially useful for extracting meaningfull features like genres or keywords, which can be used to build content based filtering models.

The convert function takes a string representation of a list of dictionaries, extracts the "name" field from each dictionary, and returns a list of those names.

In [34]:
movies.dropna(inplace=True) # The code snippet movies.dropna(inplace=True) is a method used to handle missing (NaN) 
# values in a DataFrame in pandas.

In [35]:
#movies["genres"].apply(convert)
#This applies the convert() function(explained earlier)to each row in the genres column.
#Since the genres column contains JSON-like Strings, the convert() function will:
# * convert the string into a list of dictionaries using ast.literal_eval()
# * Extract the "name" values(genres names) from the dictionaries.
# * Return a list of genres names.

import ast

def convert(value):
    if isinstance(value, str):
        try:
            return ast.literal_eval(value)
        except (ValueError, SyntaxError):
            return []  # Return an empty list if evaluation fails
    elif isinstance(value, list):
        return value  # Return the list as-is if it's already a list
    else:
        return []  # Handle other unexpected types
#If the value is a string, it tries to safely evaluate it using ast.literal_eval.
#If it’s already a list, it returns the value directly.
#If it encounters an unexpected type, it returns an empty list.
movies["genres"] = movies["genres"].apply(convert)


In [36]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [37]:
movies["keywords"] = movies["keywords"].apply(convert)
movies.head()
# apply(convert): This applies the convert function to every item in the "keywords" column.
#movies.head(): This shows the first 5 rows of the updated DataFrame, allowing you to inspect the result.

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [38]:
#Converts the string representation of a Python object (in this case, a list of dictionaries) into an actual Python object safely.
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [39]:
#What This Function Does
#Extracts at most three "name" values from a list of dictionaries stored as a string.
#Uses ast.literal_eval(text) to safely convert a string representation of a list of dictionaries into an actual Python list.
#Loops through the list and appends the "name" values to L, but only for the first three elements.
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

# step-by-step breakdown
1. initialize an empty list "L" and a counter "counter=0.
2. convert the input string(text) into a list of dictionaries using ast.literal_eval().
3. loop through the converted list and extract the "name" values, but only for the first 3 elements.
4. increment the counter after each iteration.
5. return the list "L" containing at most three names.

In [40]:
movies['cast'] = movies['cast'].apply(convert)
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{'cast_id': 242, 'character': 'Jake Sully', '...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{'cast_id': 5, 'character': 'John Carter', 'c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [41]:
#This ensures that only the first three actors from the cast list are kept.
#If a movie has more than 3 cast members, it trims the list to just the top 3.

movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

In [42]:

def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):  # Convert text to a Python list of dictionaries
        if i['job'] == 'Director':  # Check if the person's job is 'Director'
            L.append(i['name'])  # Append the director's name to the list
    return L


# what this function does
1. Extracts the director's name(s) from a crew column that contains a JSON-like string.
2. Uses ast.literal_eval() to safely convert the string into a python list.
3. Loops through the list and checks if job=="director"
4. Returns a list of director names.

In [43]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [44]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
3796,125123,Among Giants,"A manager hires Ray, off the books, to paint a...","[Comedy, Romance]","[yorkshire, love, friends]","[{'cast_id': 2, 'character': 'Raymond 'Ray'', ...",[Sam Miller]
406,51052,Arthur Christmas,"Each Christmas, Santa and his vast army of hig...","[Drama, Animation, Family, Comedy]","[holiday, santa claus, duringcreditsstinger, w...","[{'cast_id': 2, 'character': 'Arthur (voice)',...","[Barry Cook, Sarah Smith]"
3420,7347,Elite Squad,"In 1997, before the visit of the pope to Rio d...","[Drama, Action, Crime]","[slum, police brutality, brazilian, war on dru...","[{'cast_id': 12, 'character': 'Capitão Nascime...",[José Padilha]
3947,49010,Hobo with a Shotgun,A vigilante homeless man pulls into a new city...,"[Action, Comedy, Thriller]","[female nudity, prostitute, shotgun, pimp, und...","[{'cast_id': 2, 'character': 'Hobo', 'credit_i...",[Jason Eisener]
4354,5722,Beyond the Valley of the Dolls,A hip and happenin' all girl rock group head t...,"[Comedy, Drama, Thriller]","[pop, pop star, musical]","[{'cast_id': 7, 'character': 'Kelly McNamara',...",[Russ Meyer]


In [45]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [46]:
def collapse(entry):
    # Check if the entry is a string and if so, replace spaces
    if isinstance(entry, str):
        return entry.replace(" ", "")  # or any other transformation logic
    else:
        return entry  # Or handle the case where entry isn't a string

# Apply the collapse function
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

#Each column originally contains lists of names or words with spaces, and collapse() removes spaces within each entry.

In [47]:
#This code modifies the overview column by splitting each movie's description into a list of words.
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [48]:
#This code combines multiple text-based features into a single column called tags.
#It helps in building a content-based movie recommendation system.
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [49]:
#This code removes unnecessary columns after creating the tags column.
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
#new.head()

In [50]:
def flatten_and_join(x):
    # Check if x is a dictionary
    if isinstance(x, dict):
        # Flatten the dictionary by converting all values to string
        x = [str(val) for val in x.values() if isinstance(val, (str, int, float))]  # Handles only strings, integers, and floats
    # Handle cases where x is already a list or other sequence of strings
    if isinstance(x, list):
        x = [str(item) for item in x if isinstance(item, (str, int, float))]  # Ensure list elements are strings
    # Join all elements in the list into a single string
    return " ".join(x)

# Apply the function to the 'tags' column
new['tags'] = new['tags'].apply(flatten_and_join)
new.head()


,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [51]:
#This initializes a CountVectorizer, which converts text into numerical feature vectors based on word frequency (Bag-of-Words model).
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [52]:
#This converts the tags column into a numerical matrix using the Bag-of-Words (BoW) model.
vector = cv.fit_transform(new['tags']).toarray()

In [53]:
vector.shape

(4806, 5000)

* 4,800=number of movies(rows).
* 5,000=number of unique words selected by CountVectorizer(columns).

This means each movie is now represented by a vector of 5,000 word frequencies.

In [54]:
#This imports the cosine_similarity function, which helps measure how similar two movies are based on their tags vectors.

from sklearn.metrics.pairwise import cosine_similarity

In [55]:
similarity = cosine_similarity(vector)

This creates a square matrix where:
* similarity[i][j] represents how similar movie i is to movie j.
* Values range from 0(not similar) to 1(identical).

In [56]:
similarity

array([[1.        , 0.07505866, 0.03538299, ..., 0.0329154 , 0.01853442,
        0.        ],
       [0.07505866, 1.        , 0.04714045, ..., 0.0438529 , 0.        ,
        0.        ],
       [0.03538299, 0.04714045, 1.        , ..., 0.02067246, 0.04656202,
        0.        ],
       ...,
       [0.0329154 , 0.0438529 , 0.02067246, ..., 1.        , 0.06497221,
        0.03922323],
       [0.01853442, 0.        , 0.04656202, ..., 0.06497221, 1.        ,
        0.04417261],
       [0.        , 0.        , 0.        , ..., 0.03922323, 0.04417261,
        1.        ]])

In [57]:
new[new['title'] == 'The Lego Movie'].index[0]

744

* ex: The expression new[new['title'] == 'The Lego Movie'].index[0] will return 0, which is the index of the first occurrence of 'The Lego Movie' in the DataFrame.
* The expression is used to find the index of the first row in the new DataFrame where the "title" is 'The Lego Movie'.

In [58]:
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

* This function seems to be designed to recommend movies similar to a given movie. It uses a similarity matrix (presumably precomputed) to find movies that are most similar to the one specified by the movie parameter.
* The recommend(movie) function:
1. Takes a movie title as input.
2. Finds the index of that movie.
3. Computes similarity scores for all other movies in the dataset.
4. Sorts the movies based on similarity and prints the titles of the top 5 most similar movies.


# 1 Find the Index of the Given Movie
Searches for the movie title in the new DataFrame.
Retrieves its index.
# 2 Compute Similarity Scores
similarity[index] returns a list of similarity scores between the given movie and all other movies.

enumerate(similarity[index]) pairs each movie’s index with its similarity score.

sorted(..., reverse=True, key=lambda x: x[1]) sorts movies by highest similarity score.
# 3 Retrieve the Top 5 Similar Movies
Skips distances[0] because it’s the movie itself.

Retrieves and prints the titles of the top 5 most similar movies.


In [59]:
recommend('Gandhi')

Lawrence of Arabia
A Passage to India
The Wind That Shakes the Barley
Barry Lyndon
Gandhi, My Father


In [60]:
import pickle

In [61]:
pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

The pickle.dump() function is used to save (serialize) Python objects (new and similarity) into binary files (movie_list.pkl and similarity.pkl).

This process allows you to store complex Python objects, like a DataFrame or a similarity matrix, so they can be loaded (deserialized) later without recalculating or recreating them.

In [62]:
import streamlit as st
import pickle
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


1. Streamlit: This will be used to create a user interface for your application (e.g., web-based movie recommendation system).
2. Pickle: Used to load or save models, data, or objects (e.g., saving and loading a trained recommendation model or data).
3. Pandas: Used for managing and manipulating structured data, such as loading a dataset of movies or ratings.
4. Cosine Similarity: Likely used to calculate the similarity between movies (or users) based on certain features (e.g., genre, rating) for a recommendation system.

In [63]:
# Load the movie data and similarity matrix (assumed precomputed)
movies = pickle.load(open('movie_list.pkl', 'rb'))  # Contains movie titles
similarity = pickle.load(open('similarity.pkl', 'rb'))  # Cosine similarity matrix


In [64]:
def recommend(movie):
    if movie not in movies['title'].values:
        return ["Movie not found. Please try another title."]
    
    index = movies[movies['title'] == movie_list].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    recommendations = [movies.iloc[i[0]].title for i in distances[1:6]]
    return recommendations

In [65]:
# Streamlit UI
st.title("Movie Recommendation System")
movie_name = st.text_input("Enter a movie name:")

if st.button("Recommend"):
    recommendations = recommend(movie_name)
    st.write("### Recommended Movies:")
    for rec in recommendations:
        st.write(f"- {rec}")

2025-03-30 16:10:18.747 
  command:

    streamlit run D:\my_program_files\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-03-30 16:10:18.749 Session state does not function when running a script without `streamlit run`


In [66]:
movies.sample(50)

,movie_id,title,tags
1707,26320,Taking Woodstock,"The story of Elliot Tiber and his family, who ..."
3054,37931,MacGruber,Ex-special operative MacGruber (Forte) is call...
3261,9962,The Good Girl,A discount store clerk strikes up an affair wi...
1068,9981,Kicking & Screaming,Phil Weston has been unathletic his entire lif...
4669,173224,Along the Roadside,Road movie about two young people from differe...
881,39437,Beloved,After Paul D. finds his old slave friend Sethe...
4209,27455,Foolish,'Foolish' Waise is a talented comedian with a ...
3549,13173,The Ten,"Ten stories, each inspired by one of the ten c..."
4367,248774,Obvious Child,"An immature, newly unemployed comic must navig..."
2652,31668,All The Queen's Men,A mismatched team of British Special Services ...
